**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

---

Mentoría #13 - Cómo hacer un Clasificador de Pliegos todoterreno (y de otros tipos de textos) usando NLP

---
Integrantes Grupo [1|2]:
*   Bruce Wayne: bruce.w@wayne-entreprises.com
*   Alfred Pennyworth: alfred@msn.com
*   Richard Grayson: robin_132@gmail.com
---

Edición 2023


# Introducción

En esta Mentoría, trabajaremos con un conjunto de datos que comprende aproximadamente 100.000 pliegos y licitaciones de diversos organismos nacionales, tanto públicos como privados. Estos datos se obtuvieron de un sistema/servicio diseñado para monitorear oportunidades de negocios, capturar la información en una base de datos, normalizarla y, posteriormente, clasificarla para informar a los usuarios según sus áreas de interés. Los usuarios de este sistema reciben alertas automáticas cada vez que se publica una oportunidad comercial que coincide con su perfil.

## Desafío

Como parte del proceso de clasificación, los pliegos se etiquetan utilizando principalmente reglas estáticas, como palabras clave, lo cual deja margen para optimizaciones. El título de cada pliego y principalmente la descripción de los objetos que se están licitando son campos de tipo texto y escritos por personas, por lo que naturalmente presentan ambigüedades y características propias del lenguaje que llevan a que un enfoque rígido y estático de clasificación, como el actual, resulte limitado y poco eficiente.

El Desafío es utilizar las técnicas de Aprendizaje Automático, logrando así un "Clasificador" que utilice técnicas de Procesamiento de Lenguaje Natural (NLP) para clasificar de manera más eficiente en qué rubro o categoría se encuentra un pliego, basándose en su texto descriptivo y otros campos relacionados.

## Interés General

Más allá de la aplicación específica en este conjunto de datos, este problema, al estar vinculado al Procesamiento de Lenguaje Natural (PLN) y la clasificación, tiene la ventaja de poder ser utilizado posteriormente para otros tipos de contenido. De esta manera, el clasificador desarrollado podría aplicarse para categorizar libros, noticias, textos, tweets, publicaciones, artículos, etc.

## Descripción del dataset

A continuación se enumeran las diferentes variables del dataset, así como una breve descripción de su significado:

- **id**: Clave única y primaria autoincremental de la tabla;
- **cargado**: Fecha de carga del pliego;
- **idexterno**: Id del pliego en la fuente;
- **referencia**: Campo auxiliar obtenido de la fuente;
- **objeto**: Campo principal, descripción del producto o servicio objeto de la licitación;
- **rubro**: Campo de categorización disponible en la fuente. No siempre está disponible;
- **agencia**: Empresa o Ente que lanza la licitación;
- **apertura**: Fecha de apertura del pliego (vencimiento para presentarse al pliego);
- **subrubro**: Subcategoría del pliego (también obtenido desde la fuente);
- **pais**: País donde se lanza la licitación;
- **observaciones**: Campo auxiliar donde se guardan datos extra que puede variar según la fuente;
- **monto**: Monto del pliego, no siempre está publicado;
- **divisaSimboloISO**: Moneda en la que se especifica el pliego;
- **visible**: campo binario que determina si el pliego va a ser visualizado por el sistema (True/False);
- **categoría**: Tipo de pliego, categorización entre diversos tipos de pliegos (Compra Directa, Licitación Simple, Subasta, etc.);
- **fuente**: Fuente de donde se obtuvo la licitación.

# Trabajo Práctico #2

Como vimos en el primer Trabajo Práctico, en esta segunda entrega tenemos como objetivo preparar los datos para poder alimentar un modelo de predicción. Aplicaremos diferentes tipos de prácticas y estrategias para preparar el dataset y resolver todas las cuestiones detectadas en el TP1

A continuación se enumeran los ejercicios propuestos como guía:

## Ejercicio 1: Limpieza de Datos

Según lo vimos en el TP1, aplicar las técnicas propuesta para limpiar los datos. En todos los casos, además de aplicar la técnica, redactar brevemente la explicación del criterio tomado.

1.   Selección de Variables. Como vimos, hay variables que pueden descartarse.
2.   Corrección de los diferentes Tipos de Variables. Aplicar las conversiones de tipo necesarias.
3.   Selección de Registros. Filtrar las filas que no utilizaremos para la clasificación.
4.   Curación de la variable [monto]. Tip: usar la variable [divisaSimboloISO]
5.   Curación de la variable [fecha]. Incluye la eliminación de Outliers.
6.   En [agencia], normalizar si es que existen entradas con valores equivalentes.
7.   (Opcional) Para las variables categóricas aplicar técnicas de encoding, de manera de ir transformando variables de texto en números que los algoritmos puedan procesar.

## Ejercicio 2: Variable [rubro]

1.   Unificar categorías (rubros) equivalentes entre distintas fuentes. Usar un criterio definido por el grupo y explicarlo en pocas palabras.
2.   Utilizar visualizaciones que permitan ver todas las categorías y verificar que sean valores válidos.
3.   En caso de haber registros con categorías inválidas, tomar algún criterio para imputar la categoría o eliminar el registro. Desarrollar la decisión tomada.
4.   Resolver si se va a utilizar subrubro para aperturar la categorización en algún caso. Definir luego qué postura tomar con respecto a la variable [rubro]
5.   En las casos en que existe más de una categoría, tomar alguna postura para que los registros finalmente queden con una única etiqueta.

## Ejercicio 3: Variable [objeto]

1.   Aplicar técnicas de lematización para eliminar artículos, conectores, stop_words en general. Quitar acentos (tildes), mayúsculas, plurales, caractéres inválidos, abreviaturas, etc., dejando solo palabras importantes para la clasificación.
2.   Utilizar técnicas de visualización para detectar otros términos muy frecuentes que no aportan valor a la clasificación.
3.   (Opcional) Aplicar técnicas de tokenización para convertir la variable objeto en un array de tokens.

## Ejercicio 4: Exportación

Con el dataset ya transformado, generar una versión final y exportarla a un nuevo archivo. Este es el resultado final del TP2.

## Ejercicio 5: Análisis

En base a todo lo aprendido en las materias obligatorias cursadas, qué tipo de modelos de clasificación piensan que podrían aplicarse al dataset obtenido? Desarrollar brevemente una justificación en comparación con otros modelos posibles.

# Desarrollo

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', None)

df = pd.read_csv("./datos/Mentoria_Dataset_0.csv" , encoding='latin-1')

In [ ]:
#en el practico 1 decidimos tirar los datos que tienen categoría 2:
#tiro los que tienen categoria 2
df_fil = df[df['categoria']==1]

#variables que decidimos relevantes el práctico pasado: 
variables = ['referencia', 'objeto', 'rubro','agencia','apertura','monto','fuente']

#filtro el dataframe con estas variables:
df_fil = df_fil[variables]

In [ ]:
#Hacemos el filtrado de fechas hecho en el práctico anterior:

today_date = pd.Timestamp.today().date()
min_date = pd.to_datetime('13/12/2015')
df_fil.reset_index(inplace=True)  #reseteo indices, es util cuando indice=nro de fila

#fechas malas que encontramos:
fecha_mala1 = df_fil['apertura'][62]
fecha_mala2 = df_fil['apertura'][2025]
fecha_mala3 = df_fil['apertura'][11907]

#removemos las fechas malas
df_fil = df_fil.drop(df_fil[df_fil['apertura'] == fecha_mala1].index)
df_fil = df_fil.drop(df_fil[df_fil['apertura'] == fecha_mala2].index)
df_fil = df_fil.drop(df_fil[df_fil['apertura'] == fecha_mala3].index)

#convierto a formato datetime
df_fil['apertura'] = pd.to_datetime(df_fil['apertura'],format='%d/%m/%Y %H:%M')
df_fil['apertura'] = df_fil['apertura'].dt.tz_localize(None)
df_fil['apertura'] = df_fil['apertura'].dt.date

#removemos outliers
df_fil = df_fil.drop(df_fil[df_fil['apertura'] > today_date].index)
df_fil = df_fil.drop(df_fil[pd.to_datetime(df_fil['apertura']) < min_date].index)


In [ ]:
#precio del dolar oficial
dolar_value = 255

df_fil['monto'] = df_fil['monto'].replace(r'\N', np.nan)
df_fil['monto'] = df_fil['monto'].replace('0', 0)

#remuevo las strings de espacios, pesos y dolares donde las haya. Cambio la representacion de string a float
j=0
for monto, i in zip(df_fil['monto'],df_fil.index):
    if " " in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace(' ','')
    if "$" in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace('$','')
    if "." in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace('.','')
    if "," in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace(',','.')    
    if "U$S" in str(monto):
        df_fil['monto'][i] = str(df_fil['monto'][i]).replace('US','')
        #convierto a pesos y reemplazo la entrada
        df_fil['monto'][i] = float(df_fil['monto'][i]) * dolar_value  
        j+=1
    df_fil['monto'][i] = float(df_fil['monto'][i]) 
        

Empiezo a hacer el ejercicio 3:

a- Aplicar técnicas de lematización para eliminar artículos, conectores, stop_words en general. Quitar acentos (tildes), mayúsculas, plurales, caractéres inválidos, abreviaturas, etc., dejando solo palabras importantes para la clasificación.

b- Utilizar técnicas de visualización para detectar otros términos muy frecuentes que no aportan valor a la clasificación.
    
c- (Opcional) Aplicar técnicas de tokenización para convertir la variable objeto en un array de tokens.


donde aplicar la limpieza:
* objeto
* rubro
* agencia

# Ejercicio 3: Variable [objeto]

## Ejercicio 3a: limpieza

Empecemos viendo de forma general la distribución de palabras que hay en la variable **objeto**, viendo las primeras entradas del dataset para tener una idea. 

Notamos que:
* Las entradas están llenas de signos de puntiación,
* hay muchos artículos y conectores,
* muchas palabras repetidas por estar algunas en mayúsculas, otras en minúsculas, y otras pueden estar abreviadas.
* muchas palabras son las mismas y sólo cambian si están en singular y plural.

Para tener una idea más detallada, usamos la librería `wordcloud`:

In [ ]:
import string
import wordcloud

#defino una función para plotear cloudwords:
def plot_cloudword(data,name):
    entire_words = "".join(item for item in str(data))
    fig=plt.figure(figsize=(10,4))
    ax=fig.add_subplot(111)
    wc = wordcloud.WordCloud(background_color="white",max_words=500,
                          max_font_size=300,
                          width=1920, 
                          height=1080,).generate(entire_words)
    ax.imshow(wc, interpolation='bilinear')
    ax.axis("off")
    ax.margins(x=0, y=0)
    plt.savefig(name,dpi=300)
#-------------------------------------------------------------------
df_fil.dropna(inplace=True)    #sigue habiendo NaNs, los elimino
#df_fil.reset_index(inplace=True)

print(df_fil['objeto'][:100])

plot_cloudword(df_fil['objeto'],'palabras_inicio')

Veamos cuántas palabras distintas hay en este dataset inicial:

In [ ]:
entire_words = "".join(item for item in df_fil['objeto'])
list0 = entire_words.split()
dfdf_dum2 = pd.DataFrame(list0,columns=['palabras'])

counts = dfdf_dum2.value_counts()
nwords = len(dfdf_dum2)
print('cantidad de palabras en monto:',nwords)
print('cantidad de palabras distintas:',len(counts))

Lo primero que hacemos para limpiar la variable **objeto** es pasar todas las palabras a minúsculas. También cambiamos algunos espacios dobles o triples por espacios simples. Al hacer esto, pueden quedar espacios dobles o triples. Convertimos todo a espacios simples:

In [ ]:
df_fil.reset_index(drop=True,inplace=True)

table = str.maketrans('', '', string.punctuation) #tabla para eliminar signos de puntuacion

#paso todo a minúsculas
i=0
for sentence in df_fil['objeto']:    
    df_fil.loc[i,'objeto'] = str(df_fil['objeto'][i]).lower()
    i+=1
i=0
for sentence in df_fil['objeto']:    
    df_fil.loc[i,'objeto'] = str(df_fil['objeto'][i]).translate(table) #elimino signos de puntuacion
    i+=1
    
#a veces, después de eliminar algunas palabras quedan espacios dobles o triples. Cambio a un espacio:
df_fil['objeto'] = df_fil['objeto'].replace('   ',' ')
df_fil['objeto'] = df_fil['objeto'].replace('  ',' ')

#veamos cómo va quedando:
print(df_fil['objeto'][:100])
plot_cloudword(df_fil['objeto'],'palabras_min_no_punt')

Ahora que tenemos todo en minúsculas y sin signos, eliminamos tildes, artículos, preposiciones y otras palabras que creemos no aportan información relevante al pliego:

In [ ]:
articulos = ['el','la','los','las','un','uno','una','unos','unas','lo','al','del']
preposiciones = ['a', 'ante', 'bajo', 'cabe', 'con', 'contra', 'de', 'desde', 'durante',
                 'en', 'entre', 'hacia', 'hasta', 'mediante', 'para', 'por', 'segun', 
                 'sin', 'so', 'sobre', 'tras', 'versus', 'via']
otras = ['y','su','de','sus','n°']

df_fil.reset_index(drop=True,inplace=True)

# remuevo tildes y elimino palabras y algunos signos que no se eliminaron antes en un solo loop:
i=0
for sentence in df_fil['objeto']:  
    #aca saco tildes, puntos suspensivos y el caracter de grado
    sent = str(sentence)   # variable a analizar
    if "á" in sent:
        df_fil.loc[i,'objeto'] = df_fil.loc[i,'objeto'].replace('á','a')
    if "é" in sent:
        df_fil.loc[i,'objeto'] = df_fil.loc[i,'objeto'].replace('é','e')
    if "í" in sent:
        df_fil.loc[i,'objeto'] = df_fil.loc[i,'objeto'].replace('í','i')
    if "ó" in sent:
        df_fil.loc[i,'objeto'] = df_fil.loc[i,'objeto'].replace('ó','o')
    if "ú" in sent:
        df_fil.loc[i,'objeto'] = df_fil.loc[i,'objeto'].replace('ú','u')
    if "..." in sent:
        df_fil.loc[i,'objeto'] = df_fil.loc[i,'objeto'].replace('...','')
    if "°" in sent:
        df_fil.loc[i,'objeto'] = df_fil.loc[i,'objeto'].replace('°','')

        
    #ahora empiezo a eliminar palabras que no aportan nada de información.
    #separo la oración en las palabras constituyentes
    words = df_fil['objeto'][i].split()
 
    #elimino artículos
    cambie_algo = False
    for art in articulos:
        if art in words:
            words.remove(art)
            cambie_algo = True
    #elimino preposiciones
    for prep in preposiciones:
        if prep in words:
            words.remove(prep)
            cambie_algo = True
    #elimino otras
    for otra in otras:
        if otra in words:
            words.remove(otra)
            cambie_algo = True
    if cambie_algo == True:
        df_fil['objeto'][i] = " ".join(words)
    i+=1

#veamos cómo va quedando:
print(df_fil['objeto'][:100])
plot_cloudword(df_fil['objeto'],'palabras_min_no_tildes')

El dataset va quedando más limpio, aunque hay algunas palabras que se repiten en la cloudword. Aplicamos una lematización a la variable objeto usando la librería `nltk`.

En nlp, 'lematizar' es simplificar una palabra a otra que contenga la información base, como abreviar. Nos sirve para juntar palabras en distintos tiempos y personas por ejemplo.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords') #creo que estamos haciendo lo mismo que al eliminar las preposiciones con esto
nltk.download('wordnet')

#funcion para lematizar una palabra
def lemmatize_word(word):
    stemmer = SnowballStemmer('spanish')
    return stemmer.stem(word)

df_fil0 = df_fil.copy()

print('Tokenizamos las oraciones en palabras')
print()
df_fil0['objeto'] = df_fil0['objeto'].apply(word_tokenize)
print(df_fil0['objeto'][:10])
print('*'*100)
print('Removemos stopwords (son como las contracciones?)')
print()
stop_words = set(stopwords.words('spanish'))
df_fil0['objeto'] = df_fil0['objeto'].apply(lambda words: [word for word in words if word not in stop_words])
print(df_fil0['objeto'][:10])
print('*'*100)
print('Aplicamos lematización a cada palabra')
print()
df_fil0['objeto'] = df_fil0['objeto'].apply(lambda words: [lemmatize_word(word) for word in words])
print(df_fil0['objeto'][:10])
print('*'*100)

# Unimos las palabras lematizadas en cada oración en otro dataset para ver cómo queda:
df_dum = df_fil0['objeto'].apply(lambda words: ' '.join(words))
#--------------------------------------------------------------
#veamos cómo va quedando:
print(df_dum[:10])
plot_cloudword(df_dum,'palabras_min_no_tildes')

veamos las distribución de palabras. En particular nos interesa qué porcentaje de los datos
ocupan las palabras más frecuentes.

In [ ]:
entire_words = "".join(item for item in df_dum)
list0 = entire_words.split()
dfdf_dum2 = pd.DataFrame(list0,columns=['palabras'])


counts = dfdf_dum2.value_counts()
nwords = len(dfdf_dum2)
print('cantidad de palabras en monto:',nwords)
print('cantidad de palabras distintas:',len(counts))
print('las 1000 palabras más frecuentes son el '+str(round(np.sum(counts[:1000])/nwords*100,2))+'% del total')
print('las 2000 palabras más frecuentes son el '+str(round(np.sum(counts[:2000])/nwords*100,2))+'% del total')
print('las 5000 palabras más frecuentes son el '+str(round(np.sum(counts[:5000])/nwords*100,2))+'% del total')
print('las 10000 palabras más frecuentes son el '+str(round(np.sum(counts[:10000])/nwords*100,2))+'% del total')
print('las 20000 palabras más frecuentes son el '+str(round(np.sum(counts[:20000])/nwords*100,2))+'% del total')
